# Mercedes-Benz Greener Manufacturing

#### Problem Statement : Reduce the time a Mercedes-Benz spends on the test bench.

Following actions should be performed:

* If for any column(s), the variance is equal to zero, then you need to remove those variable(s).


* Check for null and unique values for test and train sets.


* Apply label encoder.


* Perform dimensionality reduction.


* Predict your test_df values using XGBoost.

In [1]:
!git clone https://github.com/Simplilearn-Edu/Machine-Learning--Projects.git

Cloning into 'Machine-Learning--Projects'...
remote: Enumerating objects: 50, done.
remote: Total 50 (delta 0), reused 0 (delta 0), pack-reused 50
Unpacking objects: 100% (50/50), done.


In [15]:
import pandas as pd
import numpy as np

In [16]:
train = pd.read_csv('train.csv')
train.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [17]:
test = pd.read_csv('test.csv')
test.head()

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0


### If for any column(s), the variance is equal to zero, then you need to remove those variable(s).

## Find cardinality of column

1. Create an empty list of 3 lists

lst = [[1],[2],[other]]

2. iterate through all the columns

3. In iterator:
    
    a. find type of all the columns
    
    b. find # of unique values 
    
        i. if unique value = 1 then the variance is constant - assign to 1
        ii. if unique value = 2 then binary column - assign to 2\
        iii. else other

4. Drop 1
    

### Apply label encoder.

In [18]:
from sklearn.preprocessing import LabelEncoder

In [20]:
for c in train.columns:
    if train[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(train[c].values) + list(test[c].values))
        train[c] = lbl.transform(list(train[c].values))
        test[c] = lbl.transform(list(test[c].values))

In [21]:
print(f'Shape of train : {train.shape} \n\n Shape of test : {test.shape}')

Shape of train : (4209, 378) 

 Shape of test : (4209, 377)


### Perform dimensionality reduction.

In [22]:
from sklearn.decomposition import PCA, FastICA

In [23]:
## PCA

pca = PCA(n_components=10,random_state=42)

pca_results_train = pca.fit_transform(train.drop(['y'],axis=1))

pca_results_test = pca.fit_transform(test)

In [24]:
# ICA

ica = FastICA(n_components=10,random_state=42)

ica_results_train = ica.fit_transform(train.drop(['y'],axis=1))

ica_results_test = ica.fit_transform(test)

In [27]:
# Append decomposition to datasets

for i in range(1,11):
    train['pca_' + str(i)] = pca_results_train[:,i-1]
    test['pca_' + str(i)] = pca_results_test[:,i-1]

    train['ica_' + str(i)] = ica_results_train[:,i-1]
    test['ica_' + str(i)] = ica_results_test[:,i-1]

y_train = train['y']
y_mean = np.mean(y_train)

### Predict your test_df values using XGBoost.

In [28]:
import xgboost as xgb

In [29]:
xgb_parama = {
    'n_trees':500,
    'eta':0.005,
    'max_depth':4,
    'subsample':0.95,
    'objective':'reg:linear',
    'eval_metric':'rmse',
    'base_score':y_mean,
    'silent':1
}

In [30]:
## Creating training and test Dmatrix

dtrain = xgb.DMatrix(train.drop(['y'],axis=1),y_train)
dtest = xgb.DMatrix(test)

/Users/vipulgaur/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/vipulgaur/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [32]:
cv_result = xgb.cv(xgb_parama,
                  dtrain,
                  num_boost_round=500, # Increase 
                   early_stopping_rounds=50,
                   verbose_eval=50,
                   show_stdv=False
                  )


[0]	train-rmse:12.6401	test-rmse:12.6387
[50]	train-rmse:11.0905	test-rmse:11.1527
[100]	train-rmse:10.0181	test-rmse:10.1487
[150]	train-rmse:9.28978	test-rmse:9.48862
[200]	train-rmse:8.8028	test-rmse:9.06727
[250]	train-rmse:8.47773	test-rmse:8.80198
[300]	train-rmse:8.25941	test-rmse:8.63741
[350]	train-rmse:8.09155	test-rmse:8.53631
[400]	train-rmse:7.95297	test-rmse:8.47684
[450]	train-rmse:7.83878	test-rmse:8.44109
[499]	train-rmse:7.73445	test-rmse:8.42257


NameError: name 'cv_results' is not defined

In [33]:
num_boost_rounds = len(cv_result)

In [35]:
## train model

model = xgb.train(dict(xgb_parama,silent=0),dtrain,num_boost_round=num_boost_rounds)

[21:53:02] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [38]:
from sklearn.metrics import r2_score

print(f'Training r2 : {r2_score(dtrain.get_label(),model.predict(dtrain))}')

Training r2 : 0.6108367775830508


## Predict your test_df values using XGBoost.

In [39]:
y_pred = model.predict(dtest)

In [40]:
test_df_pred = pd.DataFrame({'id':test['ID'].astype(np.int32),'y':y_pred})

In [41]:
test_df_pred.head()

,id,y
0,1,92.219345
1,2,99.449776
2,3,80.195320
3,4,91.756744
4,5,111.537514
